In [15]:
import numpy as np
import pandas as pd
import json
from utils import *
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, GRU, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

In [6]:
truth_df = pd.DataFrame(columns=['id','truthMedian','truthClass','truthMean'])
with open('data/truth.jsonl') as data:
    for labelobj in data:
        truth = json.loads(labelobj)
        truthlabel = {'id': truth['id'], 'truthMedian': truth['truthMedian'], 'truthClass': truth['truthClass'], 'truthMean': truth['truthMean']}
        truth_df = truth_df.append(truthlabel, ignore_index = True)
truth_df.head()  

,id,truthMedian,truthClass,truthMean
0,858464162594172928,1.000000,clickbait,1.000000
1,858462320779026433,0.000000,no-clickbait,0.133333
2,858460992073863168,0.333333,no-clickbait,0.400000
3,858459539296980995,0.333333,no-clickbait,0.266667
4,858455355948384257,0.000000,no-clickbait,0.000000


In [7]:
instances_df = pd.DataFrame(columns=['id','postText'])
with open('data/instances.jsonl') as data:
	for instanceobj in data:
		instance = json.loads(instanceobj)
		instancerow = {'id': instance['id'], 'postText': instance['postText']}
		instances_df = instances_df.append(instancerow, ignore_index=True)
instances_df.head()

,id,postText
0,858462320779026433,[UK’s response to modern slavery leaving victi...
1,858421020331560960,[this is good]
2,858368123753435136,"[The ""forgotten"" Trump roast: Relive his bruta..."
3,858323428260139008,[Meet the happiest #dog in the world!]
4,858283602626347008,[Tokyo's subway is shut down amid fears over a...


In [8]:
dataset = instances_df.join(truth_df.set_index('id'), on='id')
dataset = dataset.drop(labels='id',axis=1)
for i in range(len(dataset)):
    dataset['postText'].values[i] = dataset['postText'].values[i][0]
dataset['postText'].dropna(inplace=True)
dataset.head()

,postText,truthMedian,truthClass,truthMean
0,UK’s response to modern slavery leaving victim...,0.000000,no-clickbait,0.133333
1,this is good,1.000000,clickbait,1.000000
2,"The ""forgotten"" Trump roast: Relive his brutal...",0.333333,no-clickbait,0.466667
3,Meet the happiest #dog in the world!,1.000000,clickbait,0.933333
4,Tokyo's subway is shut down amid fears over an...,0.000000,no-clickbait,0.000000


In [9]:
# Convert the 'no-clickbait' or 'clickbait' to binary indicator
dataset['truthClass'] = dataset['truthClass'].apply(classToBinary)
# Convert floating number in 'truthMedian' column to integer
dataset['truthMedian'] = dataset['truthMedian'].apply(medianToInteger)
# Remove all punctuations and clear the text
dataset['postText'] = dataset['postText'].apply(cleanText)
dataset.head()

,postText,truthMedian,truthClass,truthMean
0,uks response to modern slavery leaving victims...,0,0,0.133333
1,this is good,3,1,1.000000
2,the forgotten trump roast relive his brutal 20...,1,0,0.466667
3,meet the happiest dog in the world,3,1,0.933333
4,tokyos subway is shut down amid fears over an ...,0,0,0.000000


In [16]:
import tensorflow as tf
import tensorflow_hub as hub
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

NameError: name 'tfhub_handle_preprocess' is not defined